In [1]:
# Import libraries.
import numpy as np
import pymc as pm

from scipy import stats
from scipy import integrate, optimize

import matplotlib.pyplot as plt

In [2]:
# British boarding school flu data
t_obs = np.arange(15)
I_obs = np.array([1, 3, 6, 25, 73, 222, 294, 258, 237, 191, 125, 69, 27, 11, 4])

N = 764
S_0 = 763
I_0 = 1
R_0 = 0

In [3]:
# SIR ODEs

# Defines the system of differential equations
# for the SIR model.
def sir(sir, t, beta, nu):
    S, I, R = sir
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - nu * I
    dRdt = nu * I
    return dSdt, dIdt, dRdt

# Returns the infected data for the SIR model with 
# inputted parameters beta and nu.
def fit_odeint(t, beta, nu):
    fit = integrate.odeint(sir, (S_0, I_0, R_0), t_obs, args = (beta, nu))
    return fit[:, 1]

In [4]:
# Solve for the optimal values via curve-fitting
curv_opt, cov = optimize.curve_fit(fit_odeint, t_obs, I_obs)

In [5]:
# Create likelihood, prior, and posterior
def likelihood(state):
    beta, nu = state
    I_fit = fit_odeint(t_obs, beta, nu)
    
    # Poisson likelihood
    #lik = stats.poisson.logpmf(I_obs, I_fit).sum()
    
    # L1 score likelihood
    #lik = -np.log(np.mean(abs(I_fit - I_obs)))
    
    # L2 score likelihood
    lik = -np.log(np.mean((I_fit - I_obs)**2))
    
    return lik

def prior(state):
    beta, nu = state
    return stats.norm.logpdf(beta, 1.5, 0.25) + stats.norm.logpdf(nu, 0.5, 0.1)

def posterior(state):
    return likelihood(state) + prior(state)

In [6]:
# Metropolis algorithm

# Proposal function
def q(state):
    return stats.norm.rvs(state, [0.1, 0.1], size = 2)

# MCMC
def metropolis(start, num_iter):
    best = np.zeros(2)
    best_lik = -np.inf
    
    chain = np.zeros((num_iter + 1, 2))
    chain[0] = start
    for i in range(num_iter):
        lik = likelihood(chain[i])
        if lik > best_lik:
            best = chain[i]
            best_lik = lik
        if i % 100 == 0:
            print(np.round(chain[i], 3), '\t', np.round(likelihood(chain[i]), 3))
        proposal = q(chain[i])
        p = np.exp(posterior(proposal) - posterior(chain[i]))
        if stats.uniform.rvs() < p:
            chain[i + 1] = proposal
        else:
            chain[i + 1] = chain[i]
    return chain, best

In [7]:
# Solve for optimal values via MCMC
chain, best = metropolis([1, 0.1], 10000)
mcmc_opt = best

[1.  0.1] 	 -11.13
[1.654 0.437] 	 -5.73
[1.772 0.418] 	 -6.915
[1.387 0.362] 	 -8.27
[1.648 0.446] 	 -5.743
[1.697 0.487] 	 -6.099
[1.736 0.439] 	 -6.287
[1.398 0.457] 	 -8.261
[1.296 0.528] 	 -9.087
[1.623 0.516] 	 -6.907
[1.768 0.458] 	 -6.527
[1.625 0.465] 	 -6.112
[1.488 0.497] 	 -7.824
[1.549 0.506] 	 -7.395
[1.461 0.57 ] 	 -8.492
[1.824 0.487] 	 -7.023
[1.69  0.484] 	 -6.056
[1.751 0.516] 	 -6.614
[1.811 0.526] 	 -7.014
[1.516 0.55 ] 	 -8.047
[1.745 0.497] 	 -6.401
[1.8   0.625] 	 -7.657
[1.628 0.486] 	 -6.406
[1.694 0.392] 	 -6.674
[1.801 0.343] 	 -8.073
[1.731 0.455] 	 -6.136
[1.575 0.429] 	 -6.419
[1.712 0.551] 	 -6.993
[1.765 0.548] 	 -6.981
[1.664 0.509] 	 -6.53
[1.768 0.503] 	 -6.605
[1.492 0.593] 	 -8.472
[1.627 0.59 ] 	 -7.758
[1.436 0.676] 	 -9.1
[1.776 0.593] 	 -7.407
[1.656 0.449] 	 -5.721
[1.564 0.612] 	 -8.252
[1.63  0.473] 	 -6.167
[1.667 0.437] 	 -5.736
[1.288 0.56 ] 	 -9.219
[1.887 0.384] 	 -8.069
[1.764 0.441] 	 -6.591
[1.615 0.45 ] 	 -6.037
[1.635 0.441] 	 -5.8

In [8]:
print('Curve-fitting:\t', curv_opt)
print('Metropolis:\t', mcmc_opt)

Curve-fitting:	 [1.66427741 0.44582665]
Metropolis:	 [1.66470569 0.44657423]
